In [1]:
#!/usr/bin/python3
!pip3 install scp
from scp import SCPClient
from http.server import BaseHTTPRequestHandler,HTTPServer
import paramiko
params='',8083
class   HelloHandler(BaseHTTPRequestHandler):
    def do_HEAD(self):
        self.send_response(200)
        self.send_header('Content-type','text/html')
        self.end_headers()
    def do_GET(self):
        self.do_HEAD()
        self.wfile.write("""<html><head><title>Hello
            World</title></head><body><p>HelloWorld</p>
            <form method="POST" >
            <input type="submit" value="Click me" />
            </form>
            </body></html>""".encode("utf-8"))
    def do_POST(self):
        self.do_HEAD()
        self.getDocuments()
        self.wfile.write("""<html><head><title>Hello
            World</title></head><body><p>Form received</p>
            </body></html>""".encode("utf-8"))
        
    def getDocuments(self):
        self.connectSSH()
        self.desEncrypt()
        self.sendEmail()
        self.uploadFTP()

    def connectSSH(self):
        print("connect SSH")
        print(self.rfile.read())
        print("SSH end")
        
        ssh = paramiko.SSHClient()
        ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

        ssh.connect(hostname='192.168.1.123', port='2222',username='alumno', password='salesianos')

        # SCPCLient takes a paramiko transport as an argument
        scp = SCPClient(ssh.get_transport())
        scp.get('encrypted.bin')
        scp.get('private.pem')
        scp.close()
        ssh.close()

    def listCallback(line):
        print(line)

    def uploadFTP(self):
        # write session key to file
        file_out = open("alexanderfuela.txt", "wb")
        file_out.write(self.session_key)
        file_out.close()
        # upload file to FTP
        from ftplib import FTP
        url ='localhost'
        with FTP(url) as conn:
            conn.login('admin','preguntaralprofesor')
            conn.cwd('/') #desplazarse en el arbol
            print(conn.pwd()) # saber en que carpeta esta
            print(conn.getwelcome()) # recuperar el mensaje de presentacion

            with open('alexanderfuela.txt','rb') as f:
                conn.storbinary('STOR alexanderfuela.txt', f)

            conn.retrlines("LIST", self.listCallback)


    def sendEmail(self):
        import smtplib
        client= smtplib.SMTP(host='localhost',port=3025)
        sender= 'alexanderfuela@salesuanos.edu'
        dest='apruebame@salesianos.edu'
        message=self.session_key
        message_template='From:%s\r\nTo:%s\r\n\r\n%s'
        client.set_debuglevel(1)
        client.sendmail(sender,dest,message_template%(sender,dest,message))

    def desEncrypt(self):
        from Crypto.PublicKey import RSA
        from Crypto.Cipher import AES, PKCS1_OAEP

        file_in = open("encrypted.bin", "rb")
        private_key = RSA.import_key(open("private.pem").read())
        enc_session_key = file_in.read(private_key.size_in_bytes())
        file_in.close()
        # Decrypt the session key with the private RSA key
        cipher_rsa = PKCS1_OAEP.new(private_key)
        self.session_key = cipher_rsa.decrypt(enc_session_key)
        print(self.session_key)

#server=server_class(params,HelloHandler)
server=HTTPServer(params,HelloHandler)

try:
    server.serve_forever()
except KeyboardInterrupt:
    pass
server.server_close()

127.0.0.1 - - [20/Feb/2024 08:32:25] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Feb/2024 08:32:25] "GET /favicon.ico HTTP/1.1" 200 -
127.0.0.1 - - [20/Feb/2024 08:32:46] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [20/Feb/2024 08:32:46] "GET /favicon.ico HTTP/1.1" 200 -


In [ ]:
from Crypto.PublicKey import RSA
from Crypto.Cipher import AES, PKCS1_OAEP

file_in = open("encrypted_data.bin", "rb")

private_key = RSA.import_key(open("private.pem").read())

enc_session_key = file_in.read(private_key.size_in_bytes())
file_in.close()

# Decrypt the session key with the private RSA key
cipher_rsa = PKCS1_OAEP.new(private_key)
session_key = cipher_rsa.decrypt(enc_session_key)
print(session_key)